In [2]:
import requests
import pandas as pd
import csv
from datetime import datetime, timedelta
import pytz
import logging
import time

### Simple example to fetch intraday stock data 
- 1 stock, 1 hour frequency
- Try for 1 day

In [48]:
# Define the URL
url = "https://data.alpaca.markets/v2/stocks/bars"

# Define the headers with your API key and secret
headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": "PKY8BZXWHLVDZ23RBZG9",
    "APCA-API-SECRET-KEY": "dIDVua0J76AwcHPKb65ZhYVNFB441XhiRELXcoee"
}
# I have to do for 15 stocks 
# March 15, 2019 to March 15, 2024
# I cant put the whole start and end date because it will go over the page
# And I'm not sure how to use the next page token
# So better to divide the API query for every week? 
# Define the query parameters
#AAPL,MSFT,NVDA,GOOGL,AMZN,JPM,V,MA,BAC,WFC,LLY,UNH,JNJ,MRK,ABBV
params = {
    "symbols": "AAPL",
    "timeframe": "1Hour",
    "start": "2019-03-15T08:00:00Z", #2019-03-15
    "end": "2019-03-16T21:45:00Z", #2019-03-26
    "limit": 1000,
    "adjustment": "raw",
    "feed": "sip",
    "sort": "asc"
}

# Make the request
response = requests.get(url, headers=headers, params=params)

In [49]:
# Print the response
data = response.json()

In [50]:
data

{'bars': {'AAPL': [{'c': 184.7,
    'h': 185.33,
    'l': 184.25,
    'n': 138,
    'o': 184.25,
    't': '2019-03-15T08:00:00Z',
    'v': 20187,
    'vw': 184.712259},
   {'c': 184.7,
    'h': 184.7,
    'l': 184.5,
    'n': 22,
    'o': 184.69,
    't': '2019-03-15T09:00:00Z',
    'v': 5491,
    'vw': 184.595864},
   {'c': 185.14,
    'h': 185.17,
    'l': 184.67,
    'n': 31,
    'o': 184.7,
    't': '2019-03-15T10:00:00Z',
    'v': 4924,
    'vw': 184.891137},
   {'c': 184.9,
    'h': 185.12,
    'l': 184.72,
    'n': 304,
    'o': 185.12,
    't': '2019-03-15T11:00:00Z',
    'v': 46489,
    'vw': 184.88643},
   {'c': 184.65,
    'h': 185.25,
    'l': 184.2,
    'n': 667,
    'o': 184.9982,
    't': '2019-03-15T12:00:00Z',
    'v': 115819,
    'vw': 184.921543},
   {'c': 184.91,
    'h': 184.99,
    'l': 183.74,
    'n': 27525,
    'o': 184.65,
    't': '2019-03-15T13:00:00Z',
    'v': 15247855,
    'vw': 184.622},
   {'c': 185.64,
    'h': 185.67,
    'l': 184.43,
    'n': 33141,


### Sample function to save file to csv 

In [91]:
def save_to_csv_test(stock, data):
    filename = f"{stock}_data.csv"
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['timestamp', 'open', 'high', 'low', 'close', 'volume', 'numtrades', 'vwap'])
        for entry in data['bars'][stock]:
            writer.writerow([
                entry['t'], entry['o'], entry['h'], entry['l'], entry['c'],
                entry['v'], entry['n'], entry['vw']
            ])

In [92]:
save_to_csv_test('AAPL', data)

## Fetching intraday of 15Min interval for 15 stocks Mar'19 - Mar'24 

In [34]:
# Function to generate weekly intervals 
# Want the start date to start at 08:00:00 UTC/04:00:00 EST 
# Want the end date to end at 21:45:00 UTC/17:45:00 EST

def generate_intervals(start, end, interval_days=7):
    intervals = []
    current_start = start
    while current_start < end:
        week_end = current_start + timedelta(days=interval_days - 1, hours=13, minutes=45)
        if week_end > end:
            week_end = end
        intervals.append((current_start, week_end))
        current_start = week_end + timedelta(days=1, hours=-13, minutes=-45)
    return intervals 

In [35]:
# Setting up logging to ensure each week interval has a succesful API request
log_filename = datetime.now().strftime('data_fetch_log_%Y%m%d_%H%M%S.log')

logging.basicConfig(
    filename=log_filename,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [36]:
# Fetch data from the API
def fetch_data(stock, start, end):
    url = "https://data.alpaca.markets/v2/stocks/bars"

    # Define the headers with your API key and secret
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": "PKY8BZXWHLVDZ23RBZG9",
        "APCA-API-SECRET-KEY": "dIDVua0J76AwcHPKb65ZhYVNFB441XhiRELXcoee"
    }

    start_str = start.isoformat().replace("+00:00", "Z")
    end_str = end.isoformat().replace("+00:00", "Z")

    params = {
        "symbols": stock,
        "timeframe": "15Min",
        "start": start_str,
        "end": end_str,
        "limit": 1000,
        "adjustment": "raw",
        "feed": "sip",
        "sort": "asc"
    }

    for attempt in range(5):
        try:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
            logging.info(f"Data fetched successfully for {stock} from {start} to {end}.")
            return response.json()
        except requests.exceptions.RequestException as e:
            if response.status_code == 429: # For too many requests
                logging.warning(f"Rate limit exceeded, retrying... (Attempt {attempt + 1})")
                time.sleep(60)
            else:
                logging.error(f"Error fetching data for {stock} from {start} to {end}: {e}")
                return []

In [37]:
def save_to_csv(stock, data, interval_num, start, end):
    start_str = start.strftime('%Y%m%d_%H%M')
    end_str = end.strftime('%Y%m%d_%H%M')
    filename = f"{stock}_data_interval_{interval_num}_{start_str}_to_{end_str}.csv"
    #filename = f"{stock}_data_{interval_num}.csv"
    try:
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['timestamp', 'open', 'high', 'low', 'close', 'volume', 'numtrades', 'vwap'])
            for entry in data['bars'][stock]:
                writer.writerow([
                    entry['t'], entry['o'], entry['h'], entry['l'], entry['c'],
                    entry['v'], entry['n'], entry['vw']
                ])
        logging.info(f"Data successfully saved to {filename}.")
    except IOError as e:
        logging.error(f"Error saving data to {filename}: {e}")

In [38]:
# Test Run
# stocks_test = ['AAPL', 'MSFT']
# START_DATE_test = datetime(2019, 3, 15, 8, 0, 0, tzinfo=pytz.UTC)
# END_DATE_test = datetime(2019, 4, 15, 21, 45, 0, tzinfo=pytz.UTC)

# Setting up the dates for API request 
START_DATE = datetime(2019, 3, 15, 8, 0, 0, tzinfo=pytz.UTC)
END_DATE = datetime(2024, 3, 15, 21, 45, 0, tzinfo=pytz.UTC)

stocks = ['AAPL', 'MSFT', 'NVDA', 'GOOGL', 'AMZN', 'JPM', 'V', 'MA', 'BAC', 'WFC', 'LLY', 'UNH', 'JNJ', 'MRK', 'ABBV']
weekly_intervals = generate_intervals(START_DATE, END_DATE)

In [51]:
# Checking how many intervals I have 
for i, (start, end) in enumerate(weekly_intervals, start=1):
    start_str = start.strftime('%Y%m%d_%H%M')
    end_str = end.strftime('%Y%m%d_%H%M')
    #print(i, start_str, end_str)

### Another test just to make sure all of the variables are correct and we can run a simple fetch using API

In [30]:
weekly_intervals[0][0].isoformat().replace("+00:00", "Z")
weekly_intervals[0][1].isoformat().replace("+00:00", "Z")

'2019-03-21T21:45:00Z'

In [26]:
stock_data = fetch_data("AAPL", weekly_intervals[0][0], weekly_intervals[0][1])

In [52]:
#stock_data

### The Actual Run 

In [40]:
for stock in stocks: 
    print(f"Processing {stock}...")
    for i, (start, end) in enumerate(weekly_intervals, start=1):
        stock_data = fetch_data(stock, start, end)
        if 'bars' in stock_data and stock in stock_data['bars']:
            save_to_csv(stock, stock_data, i, start, end)
        else:
            logging.warning(f"No data for {stock} from {start} to {end}")

Processing AAPL...
Processing MSFT...
Processing NVDA...
Processing GOOGL...
Processing AMZN...
Processing JPM...
Processing V...
Processing MA...
Processing BAC...
Processing WFC...
Processing LLY...
Processing UNH...
Processing JNJ...
Processing MRK...
Processing ABBV...


### Concatenate all csv files per stock

In [53]:
import os
import glob

In [54]:
csv_dir = 'Interval_StockData'
stocks = ['AAPL', 'MSFT', 'NVDA', 'GOOGL', 'AMZN', 'JPM', 'V', 'MA', 'BAC', 'WFC', 'LLY', 'UNH', 'JNJ', 'MRK', 'ABBV']

In [66]:
def concatenate_csvs(stock):
    pattern = os.path.join(csv_dir, f"{stock}_data_interval_*.csv")
    all_files = glob.glob(pattern)
    all_files.sort(key=lambda x: int(os.path.basename(x).split('_')[3]))
    
    df_list = []
    for filename in all_files:
        df = pd.read_csv(filename)
        df_list.append(df)
    combined_df = pd.concat(df_list, ignore_index=True)

    combined_df.to_csv(f"{stock}_intraday.csv", index=False)
    print(f"Finished merging {stock}")

In [68]:
for stock in stocks:
    concatenate_csvs(stock)

Finished merging AAPL
Finished merging MSFT
Finished merging NVDA
Finished merging GOOGL
Finished merging AMZN
Finished merging JPM
Finished merging V
Finished merging MA
Finished merging BAC
Finished merging WFC
Finished merging LLY
Finished merging UNH
Finished merging JNJ
Finished merging MRK
Finished merging ABBV
